In [1]:
########################################################################################################################
# This script organizes the feature dataset as flattened 2D data for subsequent point-prediction modeling
########################################################################################################################

In [2]:
########################################################################################################################
# Import packages
########################################################################################################################
import gc
import numpy as np
import os
import pandas as pd
import warnings
from itertools import product
from time import time
from typing import List, Literal, Optional, Union
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
########################################################################################################################
# USER_SPECIFIC SETTING
# FEAT_IN_DIR_PATH: Path of the input directory of the imputed feature datasets (created in P05_Imputation.ipynb)
# TARGET_IN_DIR_PATH: Path of the input directory storing the target datasets (created in P02_Stratified_Partitioning.ipynb)
# OUT_DIR_PATH: Path of the output directory storing the organized datasets for modeling
########################################################################################################################
FEAT_IN_DIR_PATH: str = '../00_Data/02_Processed_Data/Features/'
TARGET_IN_DIR_PATH: str = '../00_Data/02_Processed_Data/Targets/'
OUT_DIR_PATH: str = '../00_Data/02_Processed_Data/Point_Model_Data/'

In [1]:
########################################################################################################################
# USER-SPECIFIC SETTING
# Cs: Different numbers of feature encounteres to be included (Default: [1])
# Ds: Different maximum widths of the look-back window in days (Default: [60])
########################################################################################################################
Cs : list[int] = [1]
Ds : list[int] = [60]

In [2]:
########################################################################################################################
# USER-SPECIFIC SETTING
# IMPUTE_LIST: A list of strings specifying the imputation methods. (Default: ['Zero', 'Mean', 'Median'])
# Must be a non-empty sub-list of ['Zero', 'Mean', 'Median'].
########################################################################################################################
IMPUTE_LIST: list[str] = ['Zero', 'Mean', 'Median']

In [3]:
########################################################################################################################
# Define the partition of the datasets
########################################################################################################################
partitions: list[str] = ['train', 'test']

In [ ]:
########################################################################################################################
# Loop over the experiment configurations Cs and Ds, and also the partitions and imputation methods
########################################################################################################################
for conf_idx, (C, D, partition, impute) in enumerate(product(Cs,
                                                             Ds,
                                                             partitions,
                                                             IMPUTE_LIST), 1):
    log_head: str = f'[{conf_idx}. C={C}; D={D}; partition={partition}; impute={impute}] '
    if C == 1 and Ds.index(D) > 0:      # When C=1, all D values are the same
        continue

    ####################################################################################################################
    # Load the latest feature datasets created in P05_Imputation.ipynb
    ####################################################################################################################
    pat_in_path: str = os.path.join(FEAT_IN_DIR_PATH, f'{C}_encounters_{D}_days/X_Patient_{partition}_v3.parquet')
    enc_in_path: str = os.path.join(FEAT_IN_DIR_PATH, f'{C}_encounters_{D}_days/X_Encounter_{partition}_v3.parquet')
    df_pat: pd.DataFrame = pd.read_parquet(pat_in_path)
    df_enc: pd.DataFrame = pd.read_parquet(enc_in_path)
    print(f'{log_head}Feature dataset loaded with dimension = {df_pat.shape}, {df_enc.shape}')

    ####################################################################################################################
    # Load the latest target dataset created in P02_Stratified_Partitioning.ipynb
    ####################################################################################################################
    y_path: str = os.path.join(TARGET_IN_DIR_PATH, f'{C}_encounters_{D}_days_{partition}_v2.parquet')
    df_y: pd.DataFrame = pd.read_parquet(y_path)
    print(f'{log_head}Target dataset loaded with dimension = {df_y.shape}')

    ####################################################################################################################
    # Sort each dataset
    ####################################################################################################################
    id_col: str = 'PatientDurableKey'
    df_pat = df_pat.sort_values(by=id_col, ascending=True).reset_index(drop=True)
    df_enc = df_enc.sort_values(by=[id_col, 'EncDate', 'EncounterKey'], ascending=[True, True, True]).reset_index(drop=True)
    df_y = df_y.sort_values(by=id_col, ascending=True).reset_index(drop=True)
    
    ####################################################################################################################
    # Create helper columns
    ####################################################################################################################
    idx_cols: list[str] = ['PatientDurableKey', 'EncounterKey', 'EncDate']
    enc_feats: list[str] = [c for c in df_enc.columns if c not in idx_cols]

    # Encounter index and number of encounters
    df_enc_new: pd.DataFrame =  df_enc.assign(enc_slot=lambda d: d.groupby('PatientDurableKey').cumcount() + 1,    
                                              n_enc=lambda d: d.groupby('PatientDurableKey')['enc_slot'].transform('max'))
    df_enc_new['slot_r'] = (df_enc_new['enc_slot'] + (C - df_enc_new['n_enc'])).clip(1, C)

    ####################################################################################################################
    # Create multi-index for encounters and index the features by encounters
    ####################################################################################################################
    base = df_enc_new[['PatientDurableKey', 'slot_r'] + enc_feats].set_index(['PatientDurableKey', 'slot_r']).sort_index()
    df_enc_indexed = base.unstack('slot_r').reindex(columns=pd.MultiIndex.from_product([enc_feats, range(1, C+1)])).astype('float32')

    ####################################################################################################################
    # Rename the features by time-index
    ####################################################################################################################
    expected_cols: list[str] = [f'{feat}_t{slot}' for feat, slot in df_enc_indexed.columns]
    df_enc_indexed.columns = expected_cols
    df_enc_indexed = df_enc_indexed.reset_index()

    ####################################################################################################################
    # Concatenate df_enc_indexed with df_pat
    ####################################################################################################################
    df_X: pd.DataFrame = pd.merge(left=df_pat, right=df_enc_indexed, on=id_col, how='inner')
    X: np.ndarray = df_X.sort_values(by=id_col).drop(columns=id_col).to_numpy()
    print(f'{log_head}Concatenated dataset has a dimension = {X.shape}')

    ####################################################################################################################
    # Clean up df_y
    ####################################################################################################################
    y: np.ndarray = df_y['OutcomeLabel'].to_numpy()
    print(f'{log_head}Prevalence = {100*np.mean(y):.2f}%')

    ####################################################################################################################
    # Prepare the feature names
    ####################################################################################################################
    all_feats: list[str] = df_X.drop(columns=id_col).columns.tolist()
    df_feat_out: pd.DataFrame = pd.DataFrame({'Features': all_feats})

    ####################################################################################################################
    # Create a new directory and save the datasets
    ####################################################################################################################
    out_dir_s: str = os.path.join(OUT_DIR_PATH, f'{C}_encounters_{D}_days/{impute}/')
    os.makedirs(out_dir_s, exist_ok=True)
    np.save(os.path.join(out_dir_s, f'X_{partition}.npy'), X)
    np.save(os.path.join(out_dir_s, f'y_{partition}.npy'), y)
    df_feat_out.to_csv(os.path.join(out_dir_s, 'Feature_Names.csv'), index=False)
    print(f'{log_head}Packaged dataset saved in {out_dir_s}')
    print('-'*120)